# Google Playstore App Data Analysis

##### dataset source: https://www.kaggle.com/datasets/lava18/google-play-store-apps/download?datasetVersionNumber=6

## Agenda: We have google playstore data containing information of different apps installed, rating, versions and other details

### we are going to answer the following questions:
- top 10 total reviews given to the apps
- top 10 installed apps and their distribution type
- Category wise distribution of installed apps
- Top paid apps
- Top 10 paid apps with highest rating

In [2]:
# importing required libraries
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from pyspark.sql.functions import *

### Reading the file from databricks file storage

In [0]:
df = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/FileStore/tables/googleplaystore.csv")

In [0]:
df.count()

Out[10]: 10841

In [0]:
df.columns

Out[11]: ['App',
 'Category',
 'Rating',
 'Reviews',
 'Size',
 'Installs',
 'Type',
 'Price',
 'Content Rating',
 'Genres',
 'Last Updated',
 'Current Ver',
 'Android Ver']

In [0]:
df.show(1)

+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
|                 App|      Category|Rating|Reviews|Size|Installs|Type|Price|Content Rating|      Genres|   Last Updated|Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M| 10,000+|Free|    0|      Everyone|Art & Design|January 7, 2018|      1.0.0|4.0.3 and up|
+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
only showing top 1 row



In [0]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



### Dropping the unwanted columns

In [0]:
# We will drop some columns that is of no/less use in our analysis
df = df.drop("size", "content rating", "last updated", "Android Ver", "current ver")
df.show(1)

+--------------------+--------------+------+-------+--------+----+-----+------------+
|                 App|      Category|rating|reviews|installs|Type|price|      Genres|
+--------------------+--------------+------+-------+--------+----+-----+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   10000|Free|    0|Art & Design|
+--------------------+--------------+------+-------+--------+----+-----+------------+
only showing top 1 row



In [0]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- reviews: integer (nullable = true)
 |-- installs: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- Genres: string (nullable = true)



### Cleaning the data and changing the datatypes of the columns

In [0]:
# we will cast rating, reviews, installs and price column as integer
# we will regexp_replace the '+' symbol from installs and the '$' symbol from price
from pyspark.sql.functions import regexp_replace, col
df = df.withColumn('rating', col('rating').cast(DoubleType())) \
    .withColumn('reviews', col('reviews').cast(IntegerType())) \
        .withColumn('installs', regexp_replace(col('installs'),"[^0-9]","")) \
            .withColumn('installs', col('installs').cast(IntegerType())) \
                .withColumn('price', regexp_replace(col('price'),"[$]","")) \
                    .withColumn('price', col('price').cast(IntegerType()))

In [0]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- reviews: integer (nullable = true)
 |-- installs: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- Genres: string (nullable = true)



In [0]:
df.show(2)

+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|rating|reviews|installs|Type|price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   10000|Free|    0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|  500000|Free|    0|Art & Design;Pret...|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
only showing top 2 rows



### creating the temporary view 'apps' to apply sql queries on it.

In [0]:
# now change it to temporary view to apply SQL query over it.
df.createOrReplaceTempView("apps") # created a temp view with the name apps.

### top 10 highest total reviewed apps

In [0]:
%sql
select app, sum(reviews) as total_reviews from apps group by app order by total_reviews desc limit 10;

app,total_reviews
Instagram,266241989
WhatsApp Messenger,207348304
Clash of Clans,179558781
Messenger – Text and Video Chat for Free,169932272
Subway Surfers,166331958
Candy Crush Saga,156993136
Facebook,156286514
8 Ball Pool,99386198
Clash Royale,92530298
Snapchat,68045010


In [0]:
# storing the result in pyspark dataframe
top_10_review = _sqldf

In [0]:
top_10_review.show()

+--------------------+-------------+
|                 app|total_reviews|
+--------------------+-------------+
|           Instagram|    266241989|
|  WhatsApp Messenger|    207348304|
|      Clash of Clans|    179558781|
|Messenger – Text ...|    169932272|
|      Subway Surfers|    166331958|
|    Candy Crush Saga|    156993136|
|            Facebook|    156286514|
|         8 Ball Pool|     99386198|
|        Clash Royale|     92530298|
|            Snapchat|     68045010|
+--------------------+-------------+



### Top 10 highest installed apps and their distribution type

In [0]:
%sql
select app, type, sum(installs) as total_installs from apps group by app, type order by total_installs desc limit 10;

app,type,total_installs
Subway Surfers,Free,6000000000
Instagram,Free,4000000000
Google Drive,Free,4000000000
Hangouts,Free,4000000000
Google Photos,Free,4000000000
Google News,Free,4000000000
Candy Crush Saga,Free,3500000000
WhatsApp Messenger,Free,3000000000
Gmail,Free,3000000000
Temple Run 2,Free,3000000000


In [0]:
highest_install_app = _sqldf

In [0]:
highest_install_app.show()

+------------------+----+--------------+
|               app|type|total_installs|
+------------------+----+--------------+
|    Subway Surfers|Free|    6000000000|
|         Instagram|Free|    4000000000|
|      Google Drive|Free|    4000000000|
|          Hangouts|Free|    4000000000|
|     Google Photos|Free|    4000000000|
|       Google News|Free|    4000000000|
|  Candy Crush Saga|Free|    3500000000|
|WhatsApp Messenger|Free|    3000000000|
|             Gmail|Free|    3000000000|
|      Temple Run 2|Free|    3000000000|
+------------------+----+--------------+



### category wise distribution of installed apps

In [0]:
%sql
select category, sum(installs) as total_installs from apps group by category order by total_installs desc;

category,total_installs
GAME,35086024415
COMMUNICATION,32647276251
PRODUCTIVITY,14176091369
SOCIAL,14069867902
TOOLS,11452771915
FAMILY,10258263505
PHOTOGRAPHY,10088247655
NEWS_AND_MAGAZINES,7496317760
TRAVEL_AND_LOCAL,6868887146
VIDEO_PLAYERS,6222002720


In [0]:
category_apps_totals = _sqldf

In [0]:
category_apps_totals.show()

+-------------------+--------------+
|           category|total_installs|
+-------------------+--------------+
|             EVENTS|      15973161|
|             COMICS|      56086150|
|             SPORTS|    1751174498|
|            WEATHER|     426100520|
|      VIDEO_PLAYERS|    6222002720|
|  AUTO_AND_VEHICLES|      53130211|
|          PARENTING|      31521110|
|      ENTERTAINMENT|    2869160000|
|    PERSONALIZATION|    2325494782|
| HEALTH_AND_FITNESS|    1582072512|
|   TRAVEL_AND_LOCAL|    6868887146|
|BOOKS_AND_REFERENCE|    1921469576|
|     FOOD_AND_DRINK|     273898751|
|        PHOTOGRAPHY|   10088247655|
|           BUSINESS|    1001914865|
|             FAMILY|   10258263505|
|           SHOPPING|    3247848785|
|     HOUSE_AND_HOME|     168712461|
|               GAME|   35086024415|
|               Face|            42|
+-------------------+--------------+
only showing top 20 rows



### Top 10 paid apps

In [0]:
%sql
select app, sum(price) as total_price from apps where type = 'Paid' group by app order by total_price desc limit 5;

app,total_price
I'm Rich - Trump Edition,400
I Am Rich Premium,399
I am Rich,399
most expensive app (H),399
I am Rich Plus,399


In [0]:
top_installed_paid_app = _sqldf

In [0]:
top_installed_paid_app.show(truncate=False)

+------------------------+-----------+
|app                     |total_price|
+------------------------+-----------+
|I'm Rich - Trump Edition|400        |
|I Am Rich Premium       |399        |
|I am Rich               |399        |
|most expensive app (H)  |399        |
|I am Rich Plus          |399        |
+------------------------+-----------+

